# Map Grid Visualization
## Completed by:
* Denys Botuk

In [1]:
import colorsys
import folium
import geopandas as gpd
import ipynb.fs.full.ProjectAPI as api
import json
import numpy as np

from geopy.distance import geodesic

To solve the problem in a heuristic way to reduce the calculation time, I decided to observe this problem as discrete optimization problem.

The main idea is to separate the territory of Ukraine into the grid of equal square cells with placement of system in the cell center and rate each cell based on the specific characteristics according to the strategy, which we are testing at.

To make the problem more realistic, we will consider only the unoccupied territory of Ukraine. For this, we will use the following geojson and read it using geopanas library.

In [2]:
path_to_file = "map_with_war.geojson"
ukraine = gpd.read_file(path_to_file)

As it is decided to test 3 strategies, which are based on the 'critical target rate' and 'flight rate', it would be useful to create a class object 'CriticalTarget' to agregate the data about potential critical target in one object.

So, this object will contains the information about its placement (latitude and longitude), target type ('stations', 'military_objects', 'military_cities') and its critical rate. 

In [3]:
class CriticalTarget:
    def __init__(self, target_type, latitude, longitude, critical_rate):
        self.target_type = target_type
        self.latitude = latitude
        self.longitude = longitude
        self.critical_rate = critical_rate

It's reasonable to create object 'MapCell', which will contain the whole impotant information about the cell of map grid, which we will use during optimization:
* center latitude *(y coordinate of the center)*
* center longitude *(x coordinate of the center)*
* top latitude *(y coordinate of the top side)*
* bottom latitude *(y coordinate of the bottom side)*
* left longitude *(x coordinate of the left side)*
* right longitude *(x coordinate of the right side)*
* critical rate *(critical rate of cell, calculated according to the selected strategy)*
* flight rate *(percent of the air target routes, which pass through the cell, in respect to the whole number of routes)*
* critical targets rate *(percent of the critical rate sum of the critical target, that lies within the cell, with respect to the total critical rate of Ukraine)*
* strategy *(way how we calculate critical rate)*
* ppo *(air defense system, which is deployed inside the cell)*

In [4]:
class MapCell:
    def __init__(self, center_latitude, center_longitude, top_latitude, bottom_latitude, left_longitude, right_longitude, strategy, ppo):
        self.center_latitude = center_latitude
        self.center_longitude = center_longitude
        self.top_latitude = top_latitude
        self.bottom_latitude = bottom_latitude
        self.left_longitude = left_longitude
        self.right_longitude = right_longitude
        self.critical_rate = 0
        self.flight_rate = 0
        self.critical_targets_rate = 0
        self.strategy = strategy
        self.ppo = ppo
    
    def contains(self, latitude, longitude):
        '''checks if cell contains the object with coordinates "latitude" and "longitude"'''
        return self.bottom_latitude < latitude < self.top_latitude and self.left_longitude < longitude < self.right_longitude
    
    def calculate_critical_rate(self, flight_data, critical_targets):
        '''calculates critical rate of the cell according to the selected strategy'''
        # calculates critical target rate, based on the critical targets, which lie within the cell
        self.calculate_critical_targets_rate(critical_targets)
        
        # calculates, how much air targets passed through the cell in the simulated data set
        self.calculate_flight_prob(flight_data)
        
        # calculate critical rate for 'critical' strategy
        if self.strategy == 'critical':
            self.critical_rate = self.critical_targets_rate
            
        # calculate critical rate for 'route' strategy
        elif self.strategy == 'route': 
            self.critical_rate = self.flight_rate
            
        # calculate critical rate for 'combination' strategy
        elif self.strategy == 'combination':
            self.critical_rate = self.critical_targets_rate + self.flight_rate
        
    def calculate_critical_targets_rate(self, critical_targets):
        '''calculates the percentage of critical target rate with respect to the total rate within Ukraine'''
        total_rate = sum(target.critical_rate for target in critical_targets)
        self.critical_targets_rate = sum(target.critical_rate for target in critical_targets if self.contains(target.latitude, target.longitude)) / total_rate * 100
    
    def calculate_flight_prob(self, flight_data):
        '''calculates the percentage of air targets passing through the cell'''
        flight_count = 0
        for air_target in flight_data:
            for position in flight_data[air_target]:
                if self.contains(position[0], position[1]):
                    flight_count += 1
                    break
        self.flight_rate = flight_count / len(flight_data) * 100
    
    def __str__(self):
        return "radius=" + str(self.radius) \
                + "\ncenter_longitude=" + str(self.center_longitude) \
                + "\ncenter_latitude=" + str(self.center_latitude) \
                + "\ntop_latitude=" + str(self.top_latitude) \
                + "\nbottom_latitude=" + str(self.bottom_latitude) \
                + "\nleft_longitude=" + str(self.left_longitude) \
                + "\nright_longitude=" + str(self.right_longitude) \
                + "\ncritical_rate=" + str(self.critical_rate) \
                + "\nflight_rate=" + str(self.flight_rate) \
                + "\ncritical_targets_rate=" + str(self.critical_targets_rate)

To make grid more informative we need to fill each cell with appropriate color from the spectrum:

In [5]:
def spectrum(n : int):
    '''makes the spectrum of colors between yellow and red based on the count of colors'''
    hsv = [(h, 1, 1) for h in np.linspace(0, 50/360, n + 1)]
    rgb = [colorsys.hsv_to_rgb(*tup) for tup in hsv]
    defloat = lambda x: tuple((int(255 * i) for i in x))
    colors = [defloat(x) for x in rgb]
    colors.reverse()
    return colors

def rgb_to_hex(color):
    '''combines rgb color into hex'''
    r, g, b = color
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def map_to_color(colors, index):
    '''selects the color'''
    return rgb_to_hex(colors[index])

We will visualize the grid on the map, where cells are colored from yellow to red color according to its critical rate, the most red cell indicates the most critical cell, the most yellow - the least critical rate.

In [6]:
def form_tooltip(critical_rate, flight_rate, critical_targets_rate):
    '''forms the tooltip with critical rate information about cell'''
    return "critical_rate=" + str(critical_rate) + "\nflight_rate=" + str(flight_rate) + "\ncritical_targets_rate=" + str(critical_targets_rate)

def grid_visualisation(grid, strategy):
    '''makes html with the map, which contains the grid with colored cells'''
    # initialize the map
    m = folium.Map(location = [49.0139, 31.2858], zoom_start = 5.5)
    
    # calculate the maximum rate
    max_rate = int(max(map(lambda cell: cell.critical_rate if cell else 0, grid)) * 100)
    
    # get the colors spectrum
    colors = spectrum(int(max_rate))
    
    for cell in grid:
        if cell:
            folium.Rectangle(
                             [
                                (cell.bottom_latitude, cell.left_longitude),  # coordinates of the left bottom edge of the cell
                                (cell.top_latitude, cell.right_longitude) # coordinates of the right top edge of the cell
                             ], 
                             color=None,
                             fill=True, 
                             fill_color=map_to_color(colors, int(cell.critical_rate * 100)), 
                             fill_opacity=0.25,
                             tooltip=form_tooltip(cell.critical_rate, cell.flight_rate, cell.critical_targets_rate)
                            ).add_to(m)
        
    m.save(f"{strategy}/grid_map.html")

Next function creates the grid within Ukraine territory with side of specific size (in km).

If the center of the cell is not within Ukraine, the cell is set to None value (it will be useful, when we calculate critical rate for neighbor cells, such approach will allow us to make rectangle grid)

In [7]:
def makeGrid(km_shift, strategy):
    # retrives maximum and minimum x- and y-coordinates of Ukraine's border
    minx, miny, maxx, maxy = ukraine.bounds.iloc[0]
    map_cells = list()
    
    # start making grid from the left bottom point
    old_latitude = miny
    new_latitude = miny
    i = 0
    while new_latitude < maxy:
        # calculate latitude coordinate to the right from the previous cell with respect to specific value of km
        new_latitude = geodesic(kilometers=km_shift).destination((old_latitude, 0), 0).latitude
        
        old_longitude = minx
        new_longitude = minx
        j = 0
        while new_longitude < maxx:
            # calculate latitude coordinate to the top from the previous cell with respect to specific value of km
            new_longitude = geodesic(kilometers=km_shift).destination((0, old_longitude), 90).longitude
            
            # calculate coordinates of the cell's center
            center_latitude = geodesic(kilometers=km_shift / 2).destination((old_latitude, 0), 0).latitude
            center_longitude = geodesic(kilometers=km_shift / 2).destination((0, old_longitude), 90).longitude
            
            # checks if the cell's center is inside Ukraine, if not - set cell to None
            if api.is_inside_ukraine((center_latitude, center_longitude)):
                cell = MapCell(center_latitude, center_longitude, new_latitude, old_latitude, old_longitude, new_longitude, strategy, None)
                map_cells.append(cell)
            else:
                map_cells.append(None)
            
            old_longitude = new_longitude
            j += 1
        
        old_latitude = new_latitude
        i += 1
        
    
    return np.array(map_cells), (i, j)

The following function takes the information about critical targets in the following way:

target_type: [

    {
        "latitude": ...,
        "longitude": ...,
        "critical_rate": ...
    },
    ...
    
]

In [8]:
def end_points():
    '''retrieves information about critical targets'''
    with open('labeled_targets.json', 'r') as json_file:
        data = json.load(json_file)
    return data

The following function retrieves data about air target flight in the following form, which contains its coordinates in each period of time:

target type: [

    [
        latitude coordinate,
        longitude coordinate
    ],
    ...
]

In [9]:
def flight_data():
    '''retrieves flight data from the simulated data set'''
    with open('simulations.json', 'r') as json_file:
        flight_data = json.load(json_file)
    return flight_data